In [1]:
import pandas as pd
import numpy as np

#Serializer
import pickle as pkl

#Image analysis
import cv2
import mahotas as mt

#UNIX and OS functionality
import os, glob

#Vizualization
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

#sklearn models
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from sklearn import feature_selection as f_select
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix, f1_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

#Dummy Classifier
from sklearn.dummy import DummyClassifier

#Resampling algorithm
import imblearn.over_sampling
import imblearn.under_sampling

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


%matplotlib inline


In [6]:
# Image Feature Extraction
#--------------------------

def make_HUmoments(image_path):
    '''
    Hu Moments are used to characterize the outline 
    or “silhouette” of an object in an image.Use open cv2 to 
    convert the color image to a greyscale version and
    calcualte its 24 moments followed by flattening it out to 7.
    '''
    #read in the image
    img1 = cv2.imread(image_path)
    
    #Convert the image to a grayscale image
    img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    #Calculate moments from the image and flatten and reduce the moments to give HU moments
    img2_hum = cv2.HuMoments(cv2.moments(img2)).flatten()
    return img2_hum 

def image_colorfulness(image_path):
    '''
    Calculates the image colorfullness using the RGB array from
    a color image. 
    '''
    # read in image
    image = cv2.imread(image_path)
    
    # split the image into its respective RGB components
    (B, G, R) = cv2.split(image.astype("float"))

    # compute rg = R - G
    rg = np.absolute(R - G)

    # compute yb = 0.5 * (R + G) - B
    yb = np.absolute(0.5 * (R + G) - B)

    # compute the mean and standard deviation of both `rg` and `yb`
    (rbMean, rbStd) = (np.mean(rg), np.std(rg))
    (ybMean, ybStd) = (np.mean(yb), np.std(yb))

    # combine the mean and standard deviations
    stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
    meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))

    # derive the "colorfulness" metric and return it
    return stdRoot + (0.3 * meanRoot)

def get_color_ratios(image_path):
    '''
    Calculates the color ratios of each color to the total color information in image.
    '''
    # read in the image
    img1 = cv2.imread(image_path)
    
    #calculates the mean of the three channels
    mean = cv2.mean(img1)[:3]
    
    #Calculates the ratios of blue, green and red colors
    b_ratio, g_ratio, r_ratio = mean[0]/sum(mean), mean[1]/sum(mean), mean[2]/sum(mean)
    return b_ratio, g_ratio, r_ratio

def binary_ratio(image_path):
    '''
    Calculates the binary ratio of an image
    '''
     # read in the image
    img1 = cv2.imread(image_path)
    
    # convert the color image to grayscale
    img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    # Transform image to binary
    ret,thresh = cv2.threshold(img2,127,255,cv2.THRESH_BINARY_INV)
    
    # Calculate the number of black and white pixels in the image. Use
    # try and except as there are several white images that may not have white pixels 
    # pixels after transformation (these images should be completely black.
    try:
        black, white = np.unique(np.ndarray.flatten(thresh), return_counts=True)[1]
    except:
        black = np.unique(np.ndarray.flatten(thresh), return_counts=True)[1]
    
    # Calculate the binary ratio. In case the image only had black, the ratio is set to 0. 
    try:
        binary_ratio = white/(black+white)
    except:
        binary_ratio = 0
    return binary_ratio

def auto_canny(image, sigma=0.33):
    '''
    Calulates the edges of an image
    '''
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged

def make_edges(image_path):
    '''
    Uses the auto_canny function to calculate the edge ratio of an image
    '''
    # read in the image
    img1 = cv2.imread(image_path)
    
    # convert the color image to grayscale
    img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    # blur the image to quell the noise
    img2_blur = cv2.GaussianBlur(img2,(3,3), 0)
    img3 = auto_canny(img2_blur)
    
    # Calculate the number of black and white pixels in the image. Use
    # try and except as there are several white images that may not have white pixels 
    # pixels after transformation (these images should be completely black).
    try:
        black, white = np.unique(np.ndarray.flatten(img3), return_counts=True)[1]
    except:
        black = np.unique(np.ndarray.flatten(img3), return_counts=True)[1]
    
    # Calculate the edge ratio. In case the image only had black, the ratio is set to 0. 
    try:
        edges_ratio = white/(black+white)
    except:
        edges_ratio = 0
    return edges_ratio



def get_textures(image_path):
    '''
    Calculates the texture features of an image.
    '''
    # read in the image
    img1 = cv2.imread(image_path)
    
    # convert the color image to grayscale
    img2 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    # calculate haralick texture features for 4 types of adjacency
    textures = mt.features.haralick(img2)

    # take the mean of it and return it
    ht_mean = textures.mean(axis=0)
    return ht_mean
    
# Dataset for final analysis
#----------------------------

def get_features_df(base_tile_dir):
    '''
    Creates a data frame with the base_tile_dir is the directory 
    where you have stored the folders containing the image data.
    Make sure to add `from glob import glob`,`import os` `import mahotas as mt` 
    and `import opencv2` in addition to numpy and pandas. 
    '''
    dataframe = pd.DataFrame({'path': glob.glob(os.path.join(base_tile_dir, '*', '*.tif'))}) 
    dataframe['cell_type'] = dataframe['path'].map(lambda x: os.path.basename(os.path.dirname(x)).split('_')[1])
    dataframe['cell_type_idx'] = dataframe['path'].map(lambda x: os.path.basename(os.path.dirname(x)).split('_')[0])
    dataframe['image_name'] = dataframe['path'].map(lambda x: os.path.basename(x).split('_Row')[0])
    dataframe['image_color'] = dataframe['path'].map(image_colorfulness)
    dataframe['edges_ratio'] = dataframe['path'].map(make_edges)
    dataframe['binary_ratio'] = dataframe['path'].map(binary_ratio)
    dataframe['moments'] = dataframe['path'].map(make_HUmoments)
    dataframe[['moment1','moment2', 'moment3','moment4', 'moment5','moment6','moment7']] = pd.DataFrame(dataframe.moments.values.tolist(), index= dataframe.index)
    dataframe[['moment1','moment2', 'moment3','moment4']] = np.log(dataframe[['moment1','moment2', 'moment3','moment4']])
    dataframe['color_ratio'] = dataframe['path'].map(get_color_ratios)
    dataframe[['b_ratio', 'g_ratio', 'r_ratio']] = dataframe['color_ratio'].apply(pd.Series)
    dataframe['textures'] = dataframe['path'].map(get_textures)
    dataframe[['texture1','texture2', 'texture3','texture4', 'texture5','texture6','texture7', 'texture8', 'texture9','texture10', 'texture11','texture12','texture13']] = dataframe['textures'].apply(pd.Series)
    dataframe = dataframe.drop(columns =['path', 'moments','moment5','moment6','moment7', 'color_ratio', 'textures'])
    dataframe['y'] = dataframe.cell_type.map({'MUCOSA':0, 'DEBRIS':0, 'ADIPOSE':0,'EMPTY':0,'STROMA':0,'COMPLEX':0, 'LYMPHO':0,'TUMOR':1})
    return dataframe

def get_final_df(dataframe_features):
    '''
    This function takes all the x features of the model, standardizes them and carries out PCA
    on them return 2 components down from 25.
    '''
    dataframe_subset = dataframe_features.iloc[:, 'image_color':'texture13']
    ssX = StandardScaler()
    df_scaled = ssX.fit_transform(dataframe_subset)
    pca = PCA(n_components=2)
    pca.fit(df_scaled)
    pcafeatures_train = pca.transform(df_scaled)
    pca_trans = pd.DataFrame(pcafeatures_train, columns = ['PC1', 'PC2'])
    dataframe_final = dataframe_features.join(pca_trans)
    return dataframe_features
    
    
# Model Testing
#---------------

def test_logistic(X_train, y_train):
    '''This function runs Logistic Regression CV on the train dataset and returns 
       the F1 score, ROC-AUC-score and the estimator'''
    lr = LogisticRegressionCV(solver = 'lbfgs', cv =10, max_iter = 2000) 
    lr.fit(X_train, y_train)
    print('Simple Logistic Regression(lr); Train F1: %.3f, Train AUC: %.3f' % \
          (f1_score(lr.predict(X_train), y_train), roc_auc_score(y_train, lr.predict_proba(X_train)[:,1])))
    
    return lr
      
def train_oversampling(X_train, y_train, factor = 1):
    '''
    This function takes in both X_train and y_train values, calculates the imbalance in classes
    using the y_train values and applies the oversampling to both X_train and y_train. The function
    returns the oversampled train set.
    '''
    # Calculating the imbalance factor
    a = y_train.value_counts()[0]
    b = y_train.value_counts()[1]
    factor = a//b
    
    #Initializing the oversampler
    ROS = imblearn.over_sampling.RandomOverSampler(\
                                               ratio={0:a,1:b*factor}, \
                                               random_state=150) 
    
    #fitting train data and oversampling
    X_tr_rs, y_tr_rs = ROS.fit_sample(X_train, y_train)
    
    return X_tr_rs, y_tr_rs
    
def train_logistic_smote(X_train, y_train, factor = 1):
    '''
    This function takes in both X_train and y_train values, calculates the imbalance in classes
    using the y_train values and applies synthetic oversampling to both X_train and y_train. The function
    returns the oversampled train set.
    '''
    # Calculating the imbalance factor
    a = y_train.value_counts()[0]
    b = y_train.value_counts()[1]
    factor = a//b
    
    #Initializing the oversampler
    smote = imblearn.over_sampling.SMOTE(\
                                         ratio={0:a,1:b*factor}, \
                                         random_state = 150)
    
    #fitting train data and oversampling
    X_tr_smote, y_tr_smote = smote.fit_sample(X_train, y_train)

    return X_tr_smote, y_tr_smote
    
def train_logistic_undersampling(X_train, y_train, factor = 1):
    '''
    This function takes in both X_train and y_train values, calculates the imbalance in classes
    using the y_train values and applies undersampling the dominant class to both X_train and y_train. The function
    returns the oversampled train set.
    '''
    # Calculating the imbalance factor
    a = y_train.value_counts()[0]
    b = y_train.value_counts()[1]
    factor = b/a
    
    #Initializing the oversampler
    RUS = imblearn.under_sampling.RandomUnderSampler(\
                                         ratio={0:int(a*factor),1:b}, \
                                         random_state = 42)
    
    #fitting train data and oversampling
    X_tr_RUS, y_tr_RUS = RUS.fit_sample(X_train,y_train)

    return X_tr_RUS, y_tr_RUS
    
def train_logistic_weights(X_train, y_train, class_weight):
    '''
    Takes unbalanced data and balances it by adding class weights - either 
    using the in built function 'balanced' or by calculating a factor , 'factor',
    based on the actual counts of the two classes. This function is tuned more
    Binary classifications. Choose between 'balanced' or 'factor'. 
    '''
    
    if class_weight == 'balanced':
        #Model initiatlizer
        lr_balanced = LogisticRegressionCV(class_weight='balanced' \
                                           ,solver ='lbfgs', cv =10, max_iter = 2000)
        
        #Fitting models
        lr_balanced.fit(X_train, y_train)
        print('Balanced class weights Logistic Regression(lr_balanced) Test F1: %.3f, \
        Test AUC: %.3f' % \
              (f1_score(lr_balanced.predict(X_test), y_test), \
               roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))
    
    else:
        #Calculate factor 
        a = y_train.value_counts()[0]
        b = y_train.value_counts()[1]
        factor = a//b

        #Model initiatlizer
        lr_balanced = LogisticRegressionCV(class_weight={1 : factor, 0 : 1}, \
                                          solver ='lbfgs', cv=10, max_iter = 2000)

        #Fitting models
        lr_balanced.fit(X_train, y_train)
        print('Class weights Logistic Regression(lr_factorx) Test F1: %.3f, Test AUC: %.3f' % \
              (f1_score(lr_balanced.predict(X_test), y_test), \
               roc_auc_score(y_test, lr_balanced.predict_proba(X_test)[:,1])))

    return lr_balanced

def make_confusion_matrix(model, threshold = 0.5):
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    tumor_confusion = confusion_matrix(y_test, y_predict)
    
    return tumor_confusion

def best_F1_finder(lower, higher, model):
    # explicitly calling this validation since we're using it for selection
    X_val, y_val = X_test, y_test 

    thresh_ps = np.linspace(lower,higher,1000)
    # positive class probs, same basic logistic model we fit in section 2 
    model_val_probs = model.predict_proba(X_val)[:,1] 

    f1_scores = []
    for p in thresh_ps:
        model_val_labels = model_val_probs >= p
        f1_scores.append(f1_score(model_val_labels, y_val))

    plt.plot(thresh_ps, f1_scores)
    plt.title('F1 Score vs. Positive Class Decision Probability Threshold')
    plt.xlabel('P threshold')
    plt.ylabel('F1 score')

    best_f1_score = np.max(f1_scores) 
    best_thresh_p = thresh_ps[np.argmax(f1_scores)]

    print('% Model best F1 score %.3f at prob decision threshold >= %.3f' 
          % (model, best_f1_score, best_thresh_p))
    
# Visualization
#--------------

def make_confusion_matrix_graph(model, threshold=0.5):
    '''
    Uses Seaborn to draw a confusion matrix'''
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    fraud_confusion = confusion_matrix(y_test, y_predict)
    plt.figure(dpi=300)
    sns.heatmap(fraud_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['Not tumor', 'Tumor'],
           yticklabels=['Not tumor', 'Tumor']);
    plt.xticks(rotation=90)
    plt.xlabel('prediction')
    plt.ylabel('actual')
    
def make_roc_curve(list_of_models):
    plt.plot([0,1],[0,1],c='r',ls='--')
    plt.xlim([-0.05,1.05])
    plt.ylim([-0.05,1.05])
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    for model in list_of_models:
        fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
        plt.plot(fpr, tpr,lw=2)

In [2]:
base_tile_dir = './colorectal-histology-mnist/kather_texture_2016_image_tiles_5000/'

In [7]:
get_features_df(base_tile_dir)

,cell_type,cell_type_idx,image_name,image_color,edges_ratio,binary_ratio,moment1,moment2,moment3,moment4,...,texture5,texture6,texture7,texture8,texture9,texture10,texture11,texture12,texture13,y
0,TUMOR,01,10009_CRC-Prim-HE-03_009.tif,37.381346,0.271778,0.930133,-6.073384,-17.710993,-22.952445,-23.463140,...,0.101104,146.451751,4934.987736,7.798934,12.328715,0.000160,4.937310,-0.201992,0.966400,1
1,TUMOR,01,10062_CRC-Prim-HE-02_003b.tif,25.774205,0.263956,0.983600,-5.757911,-16.662870,-22.268695,-24.177842,...,0.142669,111.648998,2337.480936,7.266160,11.326210,0.000230,4.437029,-0.206366,0.959834,1
2,TUMOR,01,100B0_CRC-Prim-HE-09_009.tif,38.188592,0.240622,0.901333,-6.096181,-16.430853,-24.992701,-23.808326,...,0.107614,155.854145,5755.463773,8.046597,12.503275,0.000166,4.910681,-0.234884,0.980374,1
3,TUMOR,01,10104_CRC-Prim-HE-10_021.tif,43.204986,0.255111,0.958400,-6.024287,-17.085305,-30.175906,-25.998352,...,0.115814,135.950386,3251.260150,7.711438,12.121164,0.000184,4.807944,-0.206450,0.966795,1
4,TUMOR,01,10142_CRC-Prim-HE-09_025.tif,34.589645,0.253600,0.900356,-6.159041,-17.927785,-23.518995,-23.914664,...,0.107681,155.883556,5021.716148,7.927546,12.417574,0.000164,4.950199,-0.219488,0.974709,1
5,TUMOR,01,101A0_CRC-Prim-HE-03_034.tif,39.083090,0.278222,0.885867,-5.952663,-17.096182,-20.345515,-22.470433,...,0.106943,139.581778,7890.199085,7.876215,12.277419,0.000159,4.941289,-0.227936,0.977417,1
6,TUMOR,01,1021F_CRC-Prim-HE-04_029.tif,36.622437,0.189111,0.786489,-6.344400,-16.355852,-22.743710,-22.780591,...,0.145521,180.737948,14195.712812,8.165335,12.218724,0.000204,4.697626,-0.300559,0.992872,1
7,TUMOR,01,10264_CRC-Prim-HE-07_025.tif,44.040412,0.181156,0.743422,-6.238076,-17.443095,-22.775629,-22.957650,...,0.225979,185.258742,22550.954468,7.827969,11.463224,0.000258,4.513361,-0.331218,0.994293,1
8,TUMOR,01,10286_CRC-Prim-HE-01_002.tif,37.089954,0.177644,0.904711,-5.875066,-13.942401,-23.441924,-20.088097,...,0.249017,115.619579,10626.718049,7.404269,10.686941,0.000388,3.945506,-0.337559,0.992737,1
9,TUMOR,01,1030D_CRC-Prim-HE-02_031.tif,29.717089,0.129378,0.814000,-6.421047,-18.306516,-23.162316,-23.811809,...,0.125201,198.823585,5055.887087,8.103664,12.325967,0.000206,4.585096,-0.272132,0.988915,1
